# 📖 Text Expansion AI - Training on Novel Chapters

This notebook will:
1. Load a lightweight AI model (GPT-2 Small)
2. Fine-tune it on your markdown novel chapters
3. Generate expanded paragraphs from single sentence inputs

**No authentication required!**

## 1️⃣ Install Dependencies

In [ ]:
!pip install -q transformers peft accelerate datasets bitsandbytes
!pip install -q torch torchvision torchaudio --upgrade

print("✅ Installation complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 145.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 841.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 142.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

## 2️⃣ Import Libraries

In [ ]:
import os
import re
import torch
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    TextGenerationPipeline
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Using device: cuda
GPU: Tesla T4
Memory: 14.7 GB


## 3️⃣ Configuration

In [ ]:
# ============================================
# CONFIGURATION - Modify these as needed
# ============================================

# Model selection (all are free and don't require auth)
# Options: "gpt2", "gpt2-medium", "distilgpt2", "facebook/opt-125m"
MODEL_NAME = "gpt2"  # Smallest and fastest for training

# Training parameters
EPOCHS = 3
BATCH_SIZE = 4  # Reduce if out of memory
LEARNING_RATE = 2e-4
MAX_LENGTH = 512  # Maximum sequence length

# LoRA parameters (for efficient fine-tuning)
LORA_R = 8
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

# Output directory
OUTPUT_DIR = "./text_expansion_model"

print("Configuration loaded!")
print(f"Model: {MODEL_NAME}")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")

Configuration loaded!
Model: gpt2
Epochs: 3
Batch Size: 4


## 4️⃣ Upload Your Markdown Files

Upload your markdown files containing novel chapters. The files should contain English text.

In [ ]:
from google.colab import files

# Create directory for uploads
os.makedirs("./markdown_data", exist_ok=True)

print("📁 Please upload your markdown (.md) files:")
print("(You can select multiple files at once)")
print()

uploaded = files.upload()

# Save uploaded files
for filename, content in uploaded.items():
    filepath = f"./markdown_data/{filename}"
    with open(filepath, 'wb') as f:
        f.write(content)
    print(f"✅ Saved: {filename}")

print(f"\nTotal files uploaded: {len(uploaded)}")

### Alternative: Use Sample Data (if no files to upload)

In [ ]:
# Run this cell ONLY if you want to use sample data instead of uploading files

SAMPLE_DATA = """
# Chapter 1: The Beginning

The morning sun cast long shadows across the ancient city. Sarah stood at the edge of the marketplace, her eyes scanning the crowd for any sign of her contact. The air was thick with the scent of spices and the sound of merchants calling out their wares.

She had traveled far to reach this place, crossing mountains and rivers that few had dared to traverse. Her journey had begun three months ago, when a mysterious letter arrived at her doorstep. The letter spoke of an ancient artifact, hidden deep within the ruins of a forgotten temple.

# Chapter 2: The Discovery

Deep within the temple, Sarah discovered something that would change everything. The artifact glowed with an ethereal light, pulsing like a heartbeat in the darkness. She reached out tentatively, her fingers trembling as they approached the smooth surface.

The moment her skin made contact, visions flooded her mind. She saw civilizations rise and fall, witnessed the birth of stars and the death of worlds. Knowledge that had been lost for millennia now coursed through her consciousness.

# Chapter 3: The Journey Home

The return journey proved even more treacherous than the voyage out. Sarah clutched the artifact close to her chest, aware that countless dangers lurked in every shadow. Those who sought the artifact's power would stop at nothing to claim it.

As she walked through the moonlit forest, she could hear whispers in the darkness. The trees seemed to bend toward her, as if they too sensed the power she carried. She quickened her pace, knowing that safety was still many miles away.
"""

os.makedirs("./markdown_data", exist_ok=True)
with open("./markdown_data/sample_novel.md", "w") as f:
    f.write(SAMPLE_DATA)

print("✅ Sample data created: ./markdown_data/sample_novel.md")

## 5️⃣ Process Markdown Files

In [ ]:
def clean_markdown(text):
    """Clean markdown formatting and extract pure text."""
    # Remove markdown headers but keep the text
    text = re.sub(r'^#{1,6}\s*', '', text, flags=re.MULTILINE)
    # Remove bold/italic markers
    text = re.sub(r'\*{1,2}([^*]+)\*{1,2}', r'\1', text)
    text = re.sub(r'_{1,2}([^_]+)_{1,2}', r'\1', text)
    # Remove links but keep text
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)
    # Remove images
    text = re.sub(r'!\[([^\]]*)\]\([^)]+\)', '', text)
    # Remove code blocks
    text = re.sub(r'```[^`]*```', '', text, flags=re.DOTALL)
    text = re.sub(r'`[^`]+`', '', text)
    # Clean up whitespace
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()

def extract_paragraphs(text):
    """Extract paragraphs from text."""
    paragraphs = []
    for para in text.split('\n\n'):
        para = para.strip()
        # Only keep substantial paragraphs
        if len(para) > 50 and len(para.split()) > 10:
            paragraphs.append(para)
    return paragraphs

def create_training_pairs(paragraphs):
    """
    Create training pairs: first sentence -> full paragraph.
    This teaches the model to expand sentences into paragraphs.
    """
    training_data = []

    for para in paragraphs:
        # Extract first sentence
        sentences = re.split(r'(?<=[.!?])\s+', para)
        if len(sentences) >= 2:  # Need at least 2 sentences
            first_sentence = sentences[0].strip()
            # Create training format
            training_text = f"<|input|>{first_sentence}<|expand|>{para}<|end|>"
            training_data.append({"text": training_text})

    return training_data

# Process all markdown files
all_paragraphs = []
markdown_dir = Path("./markdown_data")

for md_file in markdown_dir.glob("*.md"):
    print(f"Processing: {md_file.name}")
    with open(md_file, 'r', encoding='utf-8') as f:
        content = f.read()

    cleaned = clean_markdown(content)
    paragraphs = extract_paragraphs(cleaned)
    all_paragraphs.extend(paragraphs)
    print(f"  Found {len(paragraphs)} paragraphs")

print(f"\nTotal paragraphs collected: {len(all_paragraphs)}")

# Create training pairs
training_data = create_training_pairs(all_paragraphs)
print(f"Training pairs created: {len(training_data)}")

# Preview a sample
if training_data:
    print("\n📝 Sample training data:")
    print("-" * 50)
    print(training_data[0]["text"][:500] + "...")

## 6️⃣ Load Model and Tokenizer

In [ ]:
print(f"Loading model: {MODEL_NAME}")
print("This may take a few minutes...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Add special tokens
special_tokens = {
    "pad_token": "<|pad|>",
    "additional_special_tokens": ["<|input|>", "<|expand|>", "<|end|>"]
}
tokenizer.add_special_tokens(special_tokens)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
)

# Resize token embeddings for new special tokens
model.resize_token_embeddings(len(tokenizer))

print(f"\n✅ Model loaded successfully!")
print(f"Parameters: {model.num_parameters():,}")

## 7️⃣ Setup LoRA for Efficient Fine-tuning

In [ ]:
# Configure LoRA (Low-Rank Adaptation)
# This allows efficient fine-tuning with much less memory

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["c_attn", "c_proj"],  # GPT-2 attention layers
    bias="none"
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\n✅ LoRA applied!")
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"Total parameters: {total_params:,}")

## 8️⃣ Prepare Dataset

In [ ]:
def tokenize_function(examples):
    """Tokenize the training data."""
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )

# Create dataset
dataset = Dataset.from_list(training_data)

# Tokenize dataset
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# Split into train and validation
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

print(f"\n✅ Dataset prepared!")
print(f"Training samples: {len(split_dataset['train'])}")
print(f"Validation samples: {len(split_dataset['test'])}")

## 9️⃣ Train the Model

In [ ]:
# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal LM, not masked LM
)

# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=100,
    learning_rate=LEARNING_RATE,
    fp16=device == "cuda",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",  # Disable wandb
    gradient_accumulation_steps=4,  # Effective batch size = 16
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=data_collator,
)

print("🚀 Starting training...")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print("-" * 50)

In [ ]:
# Train!
trainer.train()

print("\n✅ Training complete!")

## 🔟 Save the Model

In [ ]:
# Save the fine-tuned model
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Model saved to: {OUTPUT_DIR}")

✅ Model saved to: ./text_expansion_model


---

# 🎯 Text Expansion - Use Your Trained Model!

In [ ]:
def expand_text(input_sentence, max_new_tokens=200, temperature=0.8, top_p=0.9, num_return_sequences=1):
    """
    Expand a single sentence into a paragraph.

    Args:
        input_sentence: The sentence to expand
        max_new_tokens: Maximum tokens to generate
        temperature: Creativity level (0.1-1.0, higher = more creative)
        top_p: Nucleus sampling parameter
        num_return_sequences: Number of different expansions to generate

    Returns:
        List of expanded paragraphs
    """
    # Format input
    prompt = f"<|input|>{input_sentence}<|expand|>"

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt")
    if device == "cuda":
        inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            num_return_sequences=num_return_sequences,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),
            repetition_penalty=1.2
        )

    # Decode and extract expanded text
    results = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=False)
        # Extract the expanded part
        if "<|expand|>" in text:
            expanded = text.split("<|expand|>")[1]
            if "<|end|>" in expanded:
                expanded = expanded.split("<|end|>")[0]
            # Clean up special tokens
            expanded = expanded.replace("<|pad|>", "").strip()
            results.append(expanded)
        else:
            results.append(text)

    return results

print("✅ Text expansion function ready!")

✅ Text expansion function ready!


## 📝 Try It Out!

In [ ]:
# Example usage
test_sentences = [
    "The old mansion stood alone on the hill.",
    "She opened the letter with trembling hands.",
    "The storm was approaching fast."
]

print("="*60)
print("TEXT EXPANSION EXAMPLES")
print("="*60)

for sentence in test_sentences:
    print(f"\n📥 INPUT: {sentence}")
    print("-" * 40)

    expanded = expand_text(sentence, max_new_tokens=150)

    print(f"📤 OUTPUT:")
    print(expanded[0])
    print("="*60)

## 🎮 Interactive Mode

In [ ]:
# Interactive text expansion
print("="*60)
print("INTERACTIVE TEXT EXPANSION")
print("="*60)
print("Enter a sentence and get an expanded paragraph!")
print("Type 'quit' to exit.\n")

while True:
    user_input = input("📝 Enter your sentence: ")

    if user_input.lower() in ['quit', 'exit', 'q']:
        print("Goodbye! 👋")
        break

    if not user_input.strip():
        print("Please enter a valid sentence.")
        continue

    print("\n⏳ Generating...\n")
    expanded = expand_text(user_input, max_new_tokens=200)

    print("📖 EXPANDED PARAGRAPH:")
    print("-" * 40)
    print(expanded[0])
    print("\n" + "="*60 + "\n")

## 📦 Download the Trained Model

In [ ]:
# Zip and download the model
import shutil

# Create zip file
shutil.make_archive("text_expansion_model", 'zip', OUTPUT_DIR)

# Download
from google.colab import files
files.download("text_expansion_model.zip")

print("✅ Model downloaded!")

---

# 📚 Load Previously Trained Model

Use this section if you want to load a previously trained model.

In [ ]:
# Upload previously trained model (zip file)
from google.colab import files
import zipfile

print("Upload your trained model (zip file):")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('./loaded_model')
        print(f"✅ Model extracted to: ./loaded_model")

In [ ]:
# Load the saved model
from peft import PeftModel

LOADED_MODEL_PATH = "./loaded_model"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(LOADED_MODEL_PATH)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None
)
base_model.resize_token_embeddings(len(tokenizer))

# Load LoRA weights
model = PeftModel.from_pretrained(base_model, LOADED_MODEL_PATH)

print("✅ Model loaded! Ready to use.")

---

## 💡 Tips for Better Results

1. **More training data**: The more novel chapters you provide, the better the model will learn your writing style.

2. **Adjust temperature**:
   - Lower (0.3-0.5): More focused, predictable output
   - Higher (0.7-1.0): More creative, diverse output

3. **Training epochs**:
   - 3-5 epochs is usually enough
   - Too many epochs may cause overfitting

4. **Input sentences**:
   - Start with descriptive sentences
   - Include setting, character, or action details